In [9]:
# %load ir_lmart.py
#!/usr/bin/env python

# In[1]:


# Stronger baseline: Listwise L2R - LambdaMART
# Hyperparameter optimziation HPonsteroids requires Python 3!


# In[2]:


# Imports
import os
import subprocess
import sys

# REMOVE!!
from eval_utils import *

# HPO
import hpo_utils

import ConfigSpace as CS
import ConfigSpace.hyperparameters as CSH

from hpbandster.core.worker import Worker

import logging
logging.basicConfig(level=logging.WARNING)

# HPO server and stuff

# import logging
# logging.basicConfig(level=logging.WARNING)

import argparse

import hpbandster.core.nameserver as hpns
import hpbandster.core.result as hpres

from hpbandster.optimizers import BOHB as BOHB
from hpbandster.optimizers import RandomSearch as RS
from hpbandster.examples.commons import MyWorker

In [2]:
# In[3]:


# Functions
def generate_run_file(pre_run_file, run_file):
    
    with open(pre_run_file, 'rt') as input_f:
        pre_run = input_f.readlines()
#         print(type(pre_run))
    with open(run_file, 'wt') as out_f:
        for line in pre_run:
            out_f.write(line.replace('docid=','').replace('indri', 'lambdaMART'))
        


# In[4]:


# Classes
class L2Ranker:
    def __init__(self, ranklib_location, params, normalization=[]):
        self.ranklib_location = ranklib_location
        # Works with Oracle JSE
        # java version "1.8.0_211"
        # Java(TM) SE Runtime Environment (build 1.8.0_211-b12)
        # Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
        self.params = params
        self.ranker_command = ['java', '-jar', ranklib_location + 'RankLib-2.12.jar']
        self.normalization = normalization
        self.save_model_file = ''
        
#     def build(self, ir_tool_params):
    def train(self, train_data_file, save_model_file, config):
        self.save_model_file = save_model_file
        self.log_file = save_model_file + '.log'
        toolkit_parameters = [
                                *self.ranker_command, # * to unpack list elements
                                '-train',
                                train_data_file,
                                *self.normalization,
                                *self.params,
                                '-leaf', 
                                str(config['n_leaves']),
                                '-shrinkage',
                                str(config['learning_rate']),
                                '-tree', # Oner regression tree per boosted iteration
                                str(config['n_trees']),
                                '-save',
                                self.save_model_file   
                            ] 
        
        print(toolkit_parameters)
        with open(self.log_file, 'wt') as rf:
            proc = subprocess.Popen(toolkit_parameters,stdin=subprocess.PIPE, stdout=rf, stderr=subprocess.PIPE, shell=False)
#         proc = subprocess.Popen(toolkit_parameters,stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=False)
            
        (out, err)= proc.communicate()
#         print(out.decode('utf-8').splitlines())
#         print(out)

        if err == b'':
            print('Model saved: ', self.save_model_file)
        else:
#             print('error:', err, type(err))
            print('Something went wrong on training, see log: ', self.log_file)
            
  

    def gen_run_file(self, test_data_file, run_file):
        pre_run_file = run_file.replace('run_', 'pre_run_', 1)
        toolkit_parameters = [
                                *self.ranker_command, # * to unpack list elements
                                '-load',
                                self.save_model_file,
                                *self.normalization,
                                '-rank',
                                test_data_file,
                                '-indri',
                                pre_run_file     
                            ] 
        
        print(toolkit_parameters)
        with open(self.log_file, 'at') as rf:
            proc = subprocess.Popen(toolkit_parameters,stdin=subprocess.PIPE, stdout=rf, stderr=subprocess.STDOUT, shell=False)
#         proc = subprocess.Popen(toolkit_parameters,stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=False)
            
        (out, err)= proc.communicate()
#         print(out.decode('utf-8').splitlines())
#         print(out)
#         print(err)
    
        print(run_file)
        print(pre_run_file)
        
        generate_run_file(pre_run_file, run_file)
        
#         print('Run model saved: ', run_file)


# In[5]:


# try:
#     import keras
#     from keras.datasets import mnist
#     from keras.models import Sequential
#     from keras.layers import Dense, Dropout, Flatten
#     from keras.layers import Conv2D, MaxPooling2D
#     from keras import backend as K
# except:
#     raise ImportError("For this example you need to install keras.")

# try:
#     import torchvision
#     import torchvision.transforms as transforms
# except:
#     raise ImportError("For this example you need to install pytorch-vision.")

In [3]:
# Main
if __name__ == "__main__":
    
    # Options and variables
    
    parser = argparse.ArgumentParser(description='Example 1 - sequential and local execution.')
    parser.add_argument('--min_budget',   type=float, help='Minimum budget used during the optimization.',    default=2)
    parser.add_argument('--max_budget',   type=float, help='Maximum budget used during the optimization.',    default=4)
    parser.add_argument('--n_iterations', type=int,   help='Number of iterations performed by the optimizer', default=500)
    parser.add_argument('--n_workers', type=int,   help='Number of workers to run in parallel.', default=5)

#     args=parser.parse_args()
    args = fakeParser()
    
#     dataset = sys.argv[1] # 'bioasq'
#     workdir = './' + dataset + '_dir/'
#     data_split = sys.argv[2] # 'test'

    dataset = 'robust'
    workdir = './' + dataset + '_dir/'
    data_split =  'train'
    fold = 's1'  
    
    fold_dir = workdir + fold + '/'
    
    if not os.path.exists(fold_dir):
        os.makedirs(fold_dir)
        
    ranklib_location = '../../../ranklib/'
    
    
    train_data_file = fold_dir + 'rob04.train.' + fold + '_features'
    val_data_file = fold_dir + 'rob04.dev.'   +fold + '_features'
    test_data_file = fold_dir + 'rob04.test.' +fold + '_features'
    
    l2r_model = '_lmart_'
    
   
    enabled_features_file = workdir + dataset + l2r_model + 'enabled_features' # dont'f change to fold_dir!
    
#     print(enabled_features_file)
    # Train L2R model: LambdaMART
    # Parameters 
    
    n_leaves = '10'
    learning_rate = '0.1'
    n_trees = '1000'
    hpo_params = {'n_leaves': n_leaves, 'learning_rate': learning_rate, 'n_trees': n_trees}
    
    
    
    metric2t = 'MAP' # 'MAP, NDCG@k, DCG@k, P@k, RR@k, ERR@k (default=ERR@10)'
    
    ranker_type = '6' # LambdaMART
    
    # normalization: Feature Engineering?
#     norm_params = ['-norm', 'zscore'] # 'sum', 'zscore', 'linear'
    
    norm_params = ['-norm', 'zscore'] # 'sum', 'zscore', 'linear'
    
    l2r_params = [
        '-validate',
        val_data_file,
        '-ranker',
        ranker_type,
        '-metric2t',
        metric2t,
        '-feature',
        enabled_features_file
    ]
    
    # Run train
    
    lmart_model = L2Ranker(ranklib_location, l2r_params)
#     lmart_model = L2Ranker(ranklib_location, l2r_params, norm_params)
    

In [4]:
#     save_model_file = fold_dir + 'rob04.s1_lmart_model'
    
#     lmart_model.train(train_data_file, save_model_file, hpo_params)

In [5]:
#     run_file = fold_dir + 'run_' + 'rob04.s1_lmart'
    
# #     lmart_model.gen_run_file(test_data_file, run_file)

#     lmart_model.gen_run_file(val_data_file, run_file)

In [6]:
#     trec_eval_command = '../../eval/trec_eval'
#     qrels_val_file = fold_dir + 'rob04.dev.s1_qrels'
#     print(qrels_val_file)
#     print(run_file)
#     eval(trec_eval_command, qrels_val_file, run_file)

In [7]:
trec_eval_command = '../../eval/trec_eval'

def get_best_hpo(train_data_file, val_data_file, hpo_params, fold_dir, lmart_model):
    hpo_params_suffix = 'nl' + str(hpo_params['n_leaves']) + 'lr' + str(hpo_params['learning_rate']) + 'nt' + str(hpo_params['n_trees'])
    save_model_file = fold_dir + 'rob04.s1_lmart_' + hpo_params_suffix + '_model' 
    
    lmart_model.train(train_data_file, save_model_file, hpo_params)
    run_file = fold_dir + 'run_' + 'rob04.s1_lmart_' + hpo_params_suffix
    
#   lmart_model.gen_run_file(test_data_file, run_file)

    lmart_model.gen_run_file(val_data_file, run_file)
    qrels_val_file = fold_dir + 'rob04.dev.s1_qrels'
    print(qrels_val_file)
    print(run_file)
    eval_results = eval(trec_eval_command, qrels_val_file, run_file)
    eval_results = eval_results.splitlines()
    return eval_results

In [8]:
get_best_hpo(train_data_file, val_data_file, hpo_params, fold_dir, lmart_model)

['java', '-jar', '../../../ranklib/RankLib-2.12.jar', '-train', './robust_dir/s1/rob04.train.s1_features', '-validate', './robust_dir/s1/rob04.dev.s1_features', '-ranker', '6', '-metric2t', 'MAP', '-feature', './robust_dir/robust_lmart_enabled_features', '-leaf', '10', '-shrinkage', '0.1', '-tree', '1000', '-save', './robust_dir/s1/rob04.s1_lmart_nl10lr0.1nt1000_model']
Model saved:  ./robust_dir/s1/rob04.s1_lmart_nl10lr0.1nt1000_model
['java', '-jar', '../../../ranklib/RankLib-2.12.jar', '-load', './robust_dir/s1/rob04.s1_lmart_nl10lr0.1nt1000_model', '-rank', './robust_dir/s1/rob04.dev.s1_features', '-indri', './robust_dir/s1/pre_run_rob04.s1_lmart_nl10lr0.1nt1000']
./robust_dir/s1/run_rob04.s1_lmart_nl10lr0.1nt1000
./robust_dir/s1/pre_run_rob04.s1_lmart_nl10lr0.1nt1000
./robust_dir/s1/rob04.dev.s1_qrels
./robust_dir/s1/run_rob04.s1_lmart_nl10lr0.1nt1000


['map                   \tall\t0.1968',
 'P_20                  \tall\t0.3112',
 'ndcg_cut_20           \tall\t0.3689']